In [1]:
#필요한 모듈 import
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
#전체 공공자전거 고장신고 데이터 불러오기
data = pd.read_csv("/Users/janghyun/Desktop/BDP/data/대여이력 정보/data/broken.csv")

In [3]:
data.head()

,자전거번호,등록일시,고장구분
0,SPB-00108,2015.12.4 7:41,체인
1,SPB-00210,2015.12.7 12:23,체인
2,SPB-00035,2015.12.9 17:07,기타
3,SPB-01024,2015.12.13 21:02,기타
4,SPB-01015,2015.12.28 10:12,기타


In [4]:
#등록일시 type 변경
data["등록일시"]=pd.to_datetime(data['등록일시'].str.strip(), format="%Y-%m-%d %H:%M", errors='coerce')

#분류 위해 등록일시 type 변경
data["등록일시"] = data["등록일시"].dt.date
data["등록일시"] = pd.to_datetime(data["등록일시"])

data.head()

,자전거번호,등록일시,고장구분
0,SPB-00108,2015-12-04,체인
1,SPB-00210,2015-12-07,체인
2,SPB-00035,2015-12-09,기타
3,SPB-01024,2015-12-13,기타
4,SPB-01015,2015-12-28,기타


In [5]:
df1 = data.groupby(["자전거번호", "고장구분"])
df2 = df1.agg(recent = ("등록일시", np.max))
df2.columns = ["최근 고장시간"]
df2

최근 고장시간
자전거번호     고장구분           
SPB-00001 기타   2021-10-28
          단말기  2021-10-28
          안장   2019-10-05
          체인   2019-10-09
          타이어  2022-04-03
...                   ...
SPB-84985 안장   2022-05-09
SPB-84986 기타   2022-05-14
SPB-84993 기타   2022-03-29
          체인   2022-03-28
SPB-85000 타이어  2022-06-07

[187500 rows x 1 columns]

In [6]:
df3 = data.groupby(["자전거번호"])
df4 = df3.agg(recent = ("등록일시", np.max))
df4.columns = ["최근 고장시간"]
df4

,최근 고장시간
자전거번호,
SPB-00001,2022-04-03
SPB-00002,2022-06-19
SPB-00003,2022-06-04
SPB-00004,2022-06-10
SPB-00005,2021-07-14
...,...
SPB-84983,2022-05-28
SPB-84985,2022-05-09
SPB-84986,2022-05-14


In [7]:
data.head()

,자전거번호,등록일시,고장구분
0,SPB-00108,2015-12-04,체인
1,SPB-00210,2015-12-07,체인
2,SPB-00035,2015-12-09,기타
3,SPB-01024,2015-12-13,기타
4,SPB-01015,2015-12-28,기타


In [8]:
df_mean = data.groupby("자전거번호")["등록일시"].apply(lambda x: x.diff().mean().days).to_frame()
df_mean.columns=["평균 고장시간(day)"]
df_mean

,평균 고장시간(day)
자전거번호,
SPB-00001,7.0
SPB-00002,16.0
SPB-00003,13.0
SPB-00004,20.0
SPB-00005,11.0
...,...
SPB-84983,NaN
SPB-84985,NaN
SPB-84986,NaN


In [9]:
#결측치 0으로 채우기 => 한번만 고장신고된 경우
# float
df_mean.fillna(0, inplace=True)
df_mean = df_mean["평균 고장시간(day)"].astype(int).to_frame()
df_mean

,평균 고장시간(day)
자전거번호,
SPB-00001,7
SPB-00002,16
SPB-00003,13
SPB-00004,20
SPB-00005,11
...,...
SPB-84983,0
SPB-84985,0
SPB-84986,0


In [83]:
broken_data = pd.concat([df4, df_mean], axis=1)
broken_data

,최근 고장시간,평균 고장시간(day)
자전거번호,,
SPB-00001,2022-04-03,7
SPB-00002,2022-06-19,16
SPB-00003,2022-06-04,13
SPB-00004,2022-06-10,20
SPB-00005,2021-07-14,11
...,...,...
SPB-84983,2022-05-28,0
SPB-84985,2022-05-09,0
SPB-84986,2022-05-14,0


In [84]:
broken_data.describe()

,평균 고장시간(day)
count,56178.000000
mean,75.174374
std,90.261646
min,0.000000
25%,31.000000
50%,55.000000
75%,91.000000
max,2119.000000


In [266]:
broken_data.to_csv("자전거번호별_최근 고장&평균 고장시간.csv", encoding="utf-8-sig")

### 고장빈도 가장 높은 range 구하기

In [85]:
# 최빈 고장 빈도 파악 위한 range 생성
bins = np.arange(0,2151,50)

In [86]:
#series type으로 변경 및 데이터 확인
series_data = pd.Series(broken_data["평균 고장시간(day)"])
series_data

자전거번호
SPB-00001     7
SPB-00002    16
SPB-00003    13
SPB-00004    20
SPB-00005    11
             ..
SPB-84983     0
SPB-84985     0
SPB-84986     0
SPB-84993     1
SPB-85000     0
Name: 평균 고장시간(day), Length: 56178, dtype: int64

In [93]:
data_range = pd.cut(pd.Series(series_data), bins=bins, labels=bins[:-1], right=False).value_counts(sort=False) 
data_range = data_range.to_frame()
data_range.columns = ["range 50 고장 횟수"]
data_range.tail()

,range 50 고장 횟수
1900,0
1950,0
2000,0
2050,1
2100,1


In [88]:
data_range.describe()

,range 50 고장 횟수
count,43.000000
mean,1306.465116
std,4791.454399
min,0.000000
25%,1.500000
50%,6.000000
75%,67.500000
max,24899.000000


In [92]:
print(24899/56178)
print(19146/56178)
print(6526/56178)
print(2548/56178)
print(1185/56178)

0.44321620563209796
0.34080956958239883
0.1161664708604792
0.04535583324433052
0.02109366655986329
